In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier

In [14]:
input_data = pd.read_csv("train.csv", index_col='PassengerId')

def set_age(name):
    if(name.find('Mr.') != -1): #male - married or not
        return 30
    elif(name.find('Mrs.') != -1): #female - married
        return 45
    elif(name.find('Ms.') != -1): #female - married or not 
        return 30
    elif(name.find('Miss.') != -1): #female - not married 
        return 20
    else:
        return 45

def clean_dataset(dataset):
    popular_port = dataset['Embarked'].dropna().mode()[0] 
    print("This port is most popular and replace all 'NA' to " + popular_port)
    dataset['Embarked'] = dataset['Embarked'].fillna(popular_port) 
    dataset['Companions'] = dataset['SibSp'] + dataset['Parch'] #number of people travelling with
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int) #Map sex to 0-1
    embarked_one_hot = pd.get_dummies(dataset['Embarked'], prefix='Embarked') #Embarks as one-hot
    dataset = dataset.join(embarked_one_hot) 
    dataset['Age'] = dataset.apply(
                            lambda row: set_age(row['Name']) if np.isnan(row['Age']) else row['Age'], axis=1)
    
    dataset = dataset.drop(['Cabin', 'Ticket', 'Name', 'SibSp', 'Parch', 'Embarked'], axis=1) #drop useless columns
    return(dataset)
    
    
train_data = clean_dataset(input_data)
print("Prepared data:")
train_data

This port is most popular and replace all 'NA' to S
Prepared data:


,Survived,Pclass,Sex,Age,Fare,Companions,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,0,3,0,22.0,7.2500,1,0,0,1
2,1,1,1,38.0,71.2833,1,1,0,0
3,1,3,1,26.0,7.9250,0,0,0,1
4,1,1,1,35.0,53.1000,1,0,0,1
5,0,3,0,35.0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...
887,0,2,0,27.0,13.0000,0,0,0,1
888,1,1,1,19.0,30.0000,0,0,0,1
889,0,3,1,20.0,23.4500,3,0,0,1


In [15]:
# X and Y ('survived;) axis
X = train_data.drop(['Survived'], axis=1).values.astype(float)

Y = train_data['Survived'].values

def create_model(optimizer='adam', init='uniform'):
    #create model
    model = keras.Sequential()
    model.add( keras.layers.Dense(16, input_dim=X.shape[1], kernel_initializer=init, activation='relu') )
    model.add( keras.layers.Dense(8, kernel_initializer=init, activation='relu') )
    model.add( keras.layers.Dense(4, kernel_initializer=init, activation='relu') )
    model.add( keras.layers.Dense(1, kernel_initializer=init, activation='sigmoid') )

    #compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print("Model created")
    return model

In [16]:
epochs = 200
batch_size = 5
init = 'glorot_uniform'
optimizer = 'rmsprop'

#create classfier
model_predicitons = KerasClassifier(
    model=create_model, optimizer=optimizer, init=init, epochs=epochs,
    batch_size=batch_size)

model_predicitons.fit(X,Y)

Model created
Epoch 1/200
179/179 [==============================] - 1s 1ms/step - loss: 0.8340 - accuracy: 0.5275
Epoch 2/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6179 - accuracy: 0.6162
Epoch 3/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6144 - accuracy: 0.6162
Epoch 4/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6095 - accuracy: 0.6162
Epoch 5/200
179/179 [==============================] - 0s 1ms/step - loss: 0.6016 - accuracy: 0.6510
Epoch 6/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5951 - accuracy: 0.7059
Epoch 7/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5969 - accuracy: 0.7127
Epoch 8/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5836 - accuracy: 0.7250
Epoch 9/200
179/179 [==============================] - 0s 1ms/step - loss: 0.5716 - accuracy: 0.7306
Epoch 10/200
179/179 [==============================] - 0s 1ms/step - loss: 0

179/179 [==============================] - 0s 1ms/step - loss: 0.4087 - accuracy: 0.8148
Epoch 82/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4080 - accuracy: 0.8182
Epoch 83/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4142 - accuracy: 0.8238
Epoch 84/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4084 - accuracy: 0.8159
Epoch 85/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4120 - accuracy: 0.8025
Epoch 86/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4167 - accuracy: 0.8148
Epoch 87/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4127 - accuracy: 0.8204
Epoch 88/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4113 - accuracy: 0.8182
Epoch 89/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4078 - accuracy: 0.8182
Epoch 90/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4080 - accuracy: 

179/179 [==============================] - 0s 1ms/step - loss: 0.3939 - accuracy: 0.8249
Epoch 161/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3974 - accuracy: 0.8227
Epoch 162/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3973 - accuracy: 0.8159
Epoch 163/200
179/179 [==============================] - 0s 996us/step - loss: 0.3935 - accuracy: 0.8215
Epoch 164/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3986 - accuracy: 0.8316
Epoch 165/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3946 - accuracy: 0.8193
Epoch 166/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3882 - accuracy: 0.8316
Epoch 167/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3873 - accuracy: 0.8395
Epoch 168/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4023 - accuracy: 0.8227
Epoch 169/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3959 -

KerasClassifier(
	model=<function create_model at 0x000001F2FF5EA440>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=5
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=200
	init=glorot_uniform
	class_weight=None
)

In [21]:
#pred?
d = {'Pclass': [3], 'Sex': [1], 'Age': [22.0], 'Fare': [78.222], 
            'Companions': [2], 'Embarked_C': [1], 'Embarked_Q': [0], 'Embarked_S':[0]}
df = pd.DataFrame(data=d)

X_data = df.values.astype(float)

prediction = model_predicitons.predict(X_data)

print(prediction)

1/1 [==============================] - 0s 17ms/step
[1]
